In [19]:
import pandas as pd  
import numpy as np 
from collections import Counter
import patsy 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [11]:
data = pd.read_csv('bank.csv', sep = ';')
columns = ['job', 'marital','education', 'default', 'housing', 'loan','contact','poutcome','response']



for column in columns:
    counter = Counter(data[column])
    print(counter)
    


Counter({'management': 969, 'blue-collar': 946, 'technician': 768, 'admin.': 478, 'services': 417, 'retired': 230, 'self-employed': 183, 'entrepreneur': 168, 'unemployed': 128, 'housemaid': 112, 'student': 84, 'unknown': 38})
Counter({'married': 2797, 'single': 1196, 'divorced': 528})
Counter({'secondary': 2306, 'tertiary': 1350, 'primary': 678, 'unknown': 187})
Counter({'no': 4445, 'yes': 76})
Counter({'yes': 2559, 'no': 1962})
Counter({'no': 3830, 'yes': 691})
Counter({'cellular': 2896, 'unknown': 1324, 'telephone': 301})
Counter({'unknown': 3705, 'failure': 490, 'other': 197, 'success': 129})
Counter({'no': 4000, 'yes': 521})


In [14]:


bank_spec = 'response ~ age + jobtype + education + marital + default + balance + housing + loan'
filter = data['pdays'].map(lambda d: d == -1)


##jobtype
job_to_jobtype = {'admin.':'White Collar',\
    'entrepreneur':'White Collar',\
    'management':'White Collar',\
    'self-employed':'White Collar',\
    'blue-collar':'Blue Collar',\
    'services':'Blue Collar',\
    'technician':'Blue Collar'}
data['jobtype'] = data['job'].map(job_to_jobtype)
data['jobtype'] = data['jobtype'].fillna('Other/Unknown')



##response
noyes_to_binary = {'no':0, 'yes':1}
data['response'] = data['response'].map(noyes_to_binary)
data['response'] = data['response'].fillna('No')





In [15]:
# apply the filter and select columns needed for targeting model
bankwork = pd.DataFrame(data[filter], columns = ['response','age','jobtype',\
    'education',  'marital', 'default', 'balance', 'housing', 'loan'])


y_train,x_train = patsy.dmatrices(bank_spec, bankwork,return_type = 'dataframe')
print(y_train)
print(x_train)

      response
0          0.0
3          0.0
4          0.0
7          0.0
8          0.0
10         0.0
11         0.0
12         0.0
13         1.0
15         0.0
16         0.0
18         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
28         0.0
29         0.0
30         1.0
31         0.0
32         0.0
33         1.0
34         1.0
35         0.0
36         1.0
37         1.0
39         0.0
...        ...
4483       0.0
4484       1.0
4486       0.0
4487       0.0
4490       0.0
4491       0.0
4493       0.0
4494       1.0
4495       0.0
4496       0.0
4497       0.0
4498       0.0
4499       0.0
4500       0.0
4501       0.0
4502       0.0
4503       1.0
4505       1.0
4506       0.0
4507       0.0
4508       0.0
4509       0.0
4510       0.0
4511       1.0
4512       0.0
4513       0.0
4515       0.0
4516       0.0
4517       0.0
4518       0.0

[3705 rows x 1 columns]
      Intercept  jobtype[T.Other/Unknown]  jo

In [16]:
log_reg = LogisticRegression()
log_reg.fit(x_train,y_train)
log_reg.score(x_train,y_train)

D:\ANACONDA\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9090418353576248

In [23]:
print(log_reg.coef_)


y_predict = log_reg.predict(x_train)
confusion_matrix(y_train,y_predict)

[[-1.04636425e+00  5.58031699e-01  1.39057349e-01  2.89881285e-02
   2.25533953e-01 -3.56268264e-01 -5.89451153e-01 -1.15366260e-01
   1.94892399e-01 -4.09685577e-01 -6.41175431e-01  5.40429206e-03
   4.86112800e-06]]


array([[3368,    0],
       [ 337,    0]], dtype=int64)